In [ ]:
import json
import os
import requests
import boto3
from datetime import datetime
from requests.auth import HTTPBasicAuth

def lambda_handler(event, context):
    # Fetch credentials from environment variables
    client_id = os.environ.get('client_id')
    client_secret = os.environ.get('client_secret')
    
    # Define the API URL (change this to the actual API endpoint)
    uw_queue_link = "https://uwlksdjfa/1234"

    # Make the GET request with Basic Authentication
    uw_response = requests.get(uw_queue_link, auth=HTTPBasicAuth(client_id, client_secret))
    
    # Check if the request was successful
    if uw_response.status_code == 200:
        # Parse the JSON response if it returns JSON data
        uw_queue_data = uw_response.json()
    else:
        # Log the error and raise an exception if the request failed
        print(f"Failed to fetch data. Status code: {uw_response.status_code}")
        raise Exception(f"API request failed with status code {uw_response.status_code}")

    # Upload the data to S3 bucket
    client = boto3.client('s3')
    file_name = "UW_raw_" + str(datetime.now()) + ".json"
    
    # Put the data into the S3 bucket
    client.put_object(
        Bucket="unified-uw-etl-project",
        Key="raw_data/to_be_processed/" + file_name,
        Body=json.dumps(uw_queue_data)
    )
    
    # Start the AWS Glue job
    glue = boto3.client("glue")
    gluejobname = "Unified_UW_Data_Tranformation_Job"
    
    try:
        runId = glue.start_job_run(JobName=gluejobname)
        status = glue.get_job_run(JobName=gluejobname, RunId=runId['JobRunId'])
        print("Job Status : ", status['JobRun']['JobRunState'])
    except Exception as e:
        print(e)

